# Iteration 4: Fix the sampling from lognormal distributions

## Plain English Summary

In this iteration we fix the sampling from lognormal distributions.  We create a function that converts the mean and standard deviation of the lognormal to the equivalent parameters of the underlying normal distribution.  This is because `numpy` requires the parameters to be in this format.

## Notebook imports

In [1]:
from sim_tools.distributions import Lognormal

## LLM Prompt

::::{admonition} LLM Prompt
:::{include} inputs/04_iteration.md
:::
::::

## Date and time of input to LLM

> 1530 2nd March 2024

## Observations on the LLM Output

* `normal_moments_from_lognormal` is defined correctly. 

## Sources reported by the LLM

* None given - Perplexity appears to have stopped providing them.

## LLM Output

In [3]:
import simpy
import numpy as np

# External function to convert Lognormal moments to Normal moments
def normal_moments_from_lognormal(mean, std_dev):
    phi = np.sqrt(std_dev ** 2 + mean ** 2)
    mu = np.log(mean ** 2 / np.sqrt(std_dev ** 2 + mean ** 2))
    sigma = np.sqrt(np.log(phi ** 2 / mean ** 2))
    return mu, sigma

# Define the discrete-event simulation model
class AcuteStrokeUnit:
    def __init__(self, env, run_length=1825):  # 5 years in days
        self.env = env
        self.stroke_arrival = simpy.Container(env, init=0)
        self.tia_arrival = simpy.Container(env, init=0)
        self.neuro_arrival = simpy.Container(env, init=0)
        self.other_arrival = simpy.Container(env, init=0)
        self.stroke_interarrival_mean = 1.2
        self.tia_interarrival_mean = 9.3
        self.neuro_interarrival_mean = 3.6
        self.other_interarrival_mean = 3.2
        self.run_length = run_length
        self.patient_count = 0

    def stroke_acute_treatment(self, patient_type):
        if patient_type == "Rehab":
            mu, sigma = normal_moments_from_lognormal(7.4, 8.6)
            length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        elif patient_type == "ESD":
            mu, sigma = normal_moments_from_lognormal(4.6, 4.8)
            length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        else:
            mu, sigma = normal_moments_from_lognormal(7.0, 8.7)
            length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        
        yield self.env.timeout(length_of_stay)
        print(f"Stroke patient {self.patient_count} finished treatment at {self.env.now} days")

    def tia_acute_treatment(self):
        mu, sigma = normal_moments_from_lognormal(1.8, 5.0)
        length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        yield self.env.timeout(length_of_stay)
        print(f"TIA patient {self.patient_count} finished treatment at {self.env.now} days")

    def neuro_acute_treatment(self):
        mu, sigma = normal_moments_from_lognormal(4.0, 5.0)
        length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        yield self.env.timeout(length_of_stay)
        print(f"Complex Neuro patient {self.patient_count} finished treatment at {self.env.now} days")

    def other_acute_treatment(self):
        mu, sigma = normal_moments_from_lognormal(3.8, 5.2)
        length_of_stay = np.random.lognormal(mean=mu, sigma=sigma)
        yield self.env.timeout(length_of_stay)
        print(f"Other patient {self.patient_count} finished treatment at {self.env.now} days")

    def stroke_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.stroke_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.stroke_arrival.put(1)
            self.patient_count += 1
            post_asu_destination = np.random.choice(["Rehab", "ESD", "Other"], p=[0.24, 0.13, 0.63])
            print(f"Stroke patient {self.patient_count} arrived at {self.env.now} days and will go to {post_asu_destination}")
            self.env.process(self.stroke_acute_treatment(post_asu_destination))

    def tia_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.tia_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.tia_arrival.put(1)
            self.patient_count += 1
            print(f"TIA patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.tia_acute_treatment())

    def neuro_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.neuro_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.neuro_arrival.put(1)
            self.patient_count += 1
            print(f"Complex Neuro patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.neuro_acute_treatment())

    def other_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.other_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.other_arrival.put(1)
            self.patient_count += 1
            print(f"Other patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.other_acute_treatment())

# Create the simulation environment
env = simpy.Environment()

# Initialize the Acute Stroke Unit model
acu = AcuteStrokeUnit(env)

# Start the patient generators
env.process(acu.stroke_patient_generator())
env.process(acu.tia_patient_generator())
env.process(acu.neuro_patient_generator())
env.process(acu.other_patient_generator())

# Run the simulation
env.run(until=acu.run_length)



Stroke patient 1 arrived at 0.558808377890783 days and will go to Rehab
Stroke patient 2 arrived at 0.8937726271065911 days and will go to Rehab
Other patient 3 arrived at 0.9895484086085617 days
Stroke patient 3 finished treatment at 1.0111403376141004 days
Stroke patient 4 arrived at 1.0844782048643096 days and will go to Rehab
Complex Neuro patient 5 arrived at 1.2140846493772552 days
Stroke patient 6 arrived at 2.027768723018292 days and will go to Other
Stroke patient 7 arrived at 2.0882328975044873 days and will go to Other
Stroke patient 8 arrived at 2.4466728578450674 days and will go to Other
Complex Neuro patient 8 finished treatment at 2.741506183247184 days
Stroke patient 8 finished treatment at 3.2752144859850105 days
Stroke patient 9 arrived at 3.3242276853219748 days and will go to Other
Stroke patient 9 finished treatment at 3.677793234908596 days
Stroke patient 10 arrived at 3.7612512830971343 days and will go to Other
Complex Neuro patient 11 arrived at 4.137292503164

## Testing lognormal snippets

* Test of the lognormal sampling.  The code below is our own verified code. The source is given in the docstring.

In [4]:
def test_lognormal_moments(mean, std):
   
    # Lognormal class from sim-tools.
    expected_moments = Lognormal(mean, std)
    print(expected_moments.mu, expected_moments.sigma)
    
    ## llm code
    llm_mu, llm_sigma = normal_moments_from_lognormal(mean, std)
    print(llm_mu, llm_sigma)
    
    return (llm_mu, llm_sigma) == (expected_moments.mu, expected_moments.sigma)

In [5]:
test_lognormal_moments(128.79, 267.51)

4.022977459259615 1.2924439729790718
4.022977459259615 1.2924439729790718


True

## Testing

* **Extreme value test 1**: TIA, Complex Neuro, Other, have their inter-arrival time set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Stroke"
    * Actual result (PASS): The only type of patient to arrive to the model is "Accident and Emergency."
* **Extreme value test 2**: All patient types have have their inter-arrival time set to $M$ a very large number
    * Expected result: No patients arrive to the model.
    * Actual result (PASS): No patients arrive to the model.
* **Different run length**: The run length of the model is set to 10 days.
    * Expected result: The model should runs no longer than 240 simulated time periods
    * Actual result (PASS): The model runs no longer than 240 simulated time periods




### Extreme value test 1

The model uses hard coded parameters.  The most simple way to modify the code for this test was to change hard coded parameters for $M$

**Result: PASS**

In [6]:
M = 10_000_000

In [7]:
def test_1(large_number):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env)

    #set ASU arrival rates
    asu.stroke_interarrival_mean = 1.2
    asu.tia_interarrival_mean = large_number
    asu.neuro_interarrival_mean = large_number
    asu.other_interarrival_mean = large_number
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(asu.patient_count)


In [8]:
test_1(M)

Stroke patient 1 arrived at 1.298668245091576 days and will go to Rehab
Stroke patient 2 arrived at 3.6547541546307736 days and will go to Other
Stroke patient 3 arrived at 3.809373418519005 days and will go to Rehab
Stroke patient 3 finished treatment at 4.13051583060051 days
Stroke patient 4 arrived at 5.526006071485479 days and will go to Rehab
Stroke patient 5 arrived at 6.297610502723894 days and will go to Other
Stroke patient 5 finished treatment at 7.302320404919483 days
Stroke patient 6 arrived at 8.144911993999003 days and will go to Other
Stroke patient 7 arrived at 8.240569797662852 days and will go to Rehab
Stroke patient 8 arrived at 8.60438095531261 days and will go to Other
Stroke patient 9 arrived at 9.1803361553309 days and will go to Other
Stroke patient 10 arrived at 9.875589899402799 days and will go to Other
Stroke patient 10 finished treatment at 9.978253039473609 days
Stroke patient 11 arrived at 10.406450225991367 days and will go to Other
Stroke patient 11 fin

### Extreme value test 2

* The code is again modified so that all patient typs have a mean inter-arrival rate of $M$

**Result: PASS**

In [9]:
def test_2(large_number):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env)

    #set ASU arrival rates
    asu.stroke_interarrival_mean = large_number
    asu.tia_interarrival_mean = large_number
    asu.neuro_interarrival_mean = large_number
    asu.other_interarrival_mean = large_number
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(asu.patient_count)


In [10]:
test_2(M)

0


### Different run length test

The model has again been modified to use the original code and parameters generated by the LLM.

In [11]:
TEN_DAYS = 10

In [12]:
def test_3(run_length):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env, run_length=run_length)
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(env.now)
    


In [13]:
test_3(TEN_DAYS)

Stroke patient 1 arrived at 0.40319085395352106 days and will go to Other
Other patient 2 arrived at 0.6844236914865895 days
Stroke patient 3 arrived at 1.3003081144467525 days and will go to Other
Stroke patient 4 arrived at 1.9476162330253288 days and will go to Rehab
Stroke patient 5 arrived at 2.1125699003410547 days and will go to Other
Complex Neuro patient 6 arrived at 2.357239312952614 days
Stroke patient 7 arrived at 2.471175219034966 days and will go to ESD
Stroke patient 8 arrived at 2.5315218752846893 days and will go to Rehab
TIA patient 9 arrived at 2.6118747031799687 days
Other patient 9 finished treatment at 2.6140066336696695 days
Stroke patient 10 arrived at 2.6661232215178754 days and will go to Other
Stroke patient 11 arrived at 2.6888900923647454 days and will go to Rehab
TIA patient 11 finished treatment at 2.7842428033078677 days
TIA patient 12 arrived at 2.8497045006294455 days
TIA patient 12 finished treatment at 3.4688969408499055 days
Stroke patient 12 finish